## Subject set creation pipeline
This notebook demonstrates the pipeline used for creating the subject set. It includes the following codes:<br>
- get_images_from_url
- run_inference_graph_images
- load_image_into_numpy_array
- run_inference_for_single_image

These functions have to be arranged in modules so that it can be imported for further use. The modules with these functions are available [here](https://github.com/Manish-rai21bit/camera-trap-detection/tree/master/snapshot-safari)

This work is a streamlined version of [PredictAndEvaluate](https://github.com/Manish-rai21bit/camera-trap-detection/blob/master/training_demo/PredictAndEvaluation.ipynb)

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("/Users/manishrai/Desktop/UMN/Research/Zooniverse/Code/tensorflow/models/research")
from object_detection.utils import ops as utils_ops

# Importing the object detection utils
# from utils import label_map_util
# from utils import visualization_utils as vis_util

/usr/local/Cellar/python/3.7.0/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


In [2]:
import urllib, urllib.request
import os, ssl
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
# from run_inference_graph import load_image_into_numpy_array, run_inference_for_single_image

/Users/manishrai/Desktop/UMN/Research/Zooniverse/Code/tensorflow/models/research/object_detection/utils/visualization_utils.py:25: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/local/Cellar/python/3.7.0/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python/3.7.0/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_

In [3]:
%matplotlib inline

In [4]:
"""The set of functions here will help in creating the subject set. 
Subject set creation for bounding box requires the following functions:
    get_images_from_url - Download the images from the url's given into a local path
    run_inference_graph_images on the downloaded images"""

def get_images_from_url(dataset, image_name_index, url_col_index, outpath):
    """This function takes in a dataframe and downloads the images from the url's in the column.
    arguments:
        dataset - dataframe
        image_name_index - index of the column containing the image id (capture event id)
        url_col_index - index of the columns containing the url for the image id
        outpath - path on the local directory where the image has to be saved
    return: 
        Downloaded images in the path - outpath
    Usage:
        get_images_from_url(df3, image_name_index=0, url_col_index=6, outpath = '/Users/manishrai/Desktop/test_dir/')"""
    if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
        getattr(ssl, '_create_unverified_context', None)): 
        ssl._create_default_https_context = ssl._create_unverified_context
        
        check = []
        
        for i in range(dataset.shape[0]):
            if dataset.iloc[i][image_name_index] not in check:
                j = 0
            if dataset.iloc[i][image_name_index] in check:
                j += 1 
            
            print('Processing image: %d' % i)
            
            urllib.request.urlretrieve(dataset.iloc[i][url_col_index], outpath+'{0}.jpg'.format(dataset.iloc[i][image_name_index] ))
            
def run_inference_graph_images(PATH_TO_FROZEN_GRAPH, PATH_TO_LABELS, \
                               NUM_CLASSES, TEST_IMAGE_PATHS, min_threshold, \
                               bb_outpath, PATH_TO_BB_HASHMAP):
    """This function takes in a list of image local-paths and runs it through the trained graph.
    Further, using the visualization function it draws bounding boxes on each of the images and 
    saves it in a local path.
    Arguments:
        PATH_TO_FROZEN_GRAPH - local path of the trained frozen graph, '/frozen_inference_graph.pb'
        PATH_TO_LABELS - local path of the labels (a mapping from class number to class name), 'label_map_focus.pbtxt'
        NUM_CLASSES - number of detection classes
        TEST_IMAGE_PATHS - list of test image local paths
        min_threshold - minimum score threshold for the bounding box to be considered
        bb_outpath - local path where to save the images with ounding poxes, /home/ubuntu/data/tensorflow/my_workspace/camera-trap-detection/snapshot-safari/snapshot-serengeti/subject_set_upload/
        PATH_TO_BB_HASHMAP - path where bounding box information for the subjects is saved
        
        """
    # Loading the frozen graph
    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')
            
    label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
    categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    
    bb_hashmap = {}
    for image_path in TEST_IMAGE_PATHS:
        image = Image.open(image_path)
        if (len(np.array(image).shape) == 3):
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            # Considering the default dpi of matplotlib, calculating the figure size to save
            y0, x0, c = image_np.shape
            h = y0/72 # the default for dpi for matplotlib is 72
            w = x0/72 # the default for dpi for matplotlib is 72
            IMAGE_SIZE = (w, h)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Actual detection.
            output_dict = run_inference_for_single_image(image_np, detection_graph)
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
              image_np,
              output_dict['detection_boxes'],
              output_dict['detection_classes'],
              output_dict['detection_scores'],
              category_index,
              instance_masks=output_dict.get('detection_masks'),
              use_normalized_coordinates=True,
              line_thickness=8,
              min_score_thresh=min_threshold,
              skip_labels=True,
              skip_scores=True,
              agnostic_mode=True
            )
            # make a figure without the frame
            fig = plt.figure(frameon=False, figsize=IMAGE_SIZE)
            # make the content fill the whole figure
            ax = plt.Axes(fig, [0., 0., 1., 1.])
            ax.set_axis_off()
            fig.add_axes(ax)
            # draw your image
            ax.imshow(image_np)
            # saving image with boxes on the disk
            plt.savefig(os.path.join(bb_outpath, '{0}'.format(image_path[-14:])))
            plt.gcf().clear()
            bb_hashmap[image_path[-14:]] = {
              'detection_boxes' : output_dict['detection_boxes'] [0:sum(output_dict['detection_scores']>=min_threshold)],
              'detection_scores' : output_dict['detection_scores'][0:sum(output_dict['detection_scores']>=min_threshold)]
          }
            
    with open(PATH_TO_BB_HASHMAP, 'w') as f:
        for key in bb_hashmap.keys():
            f.write("%s,%s\n"%(key,bb_hashmap[key]))  

    return bb_hashmap

In [5]:
def load_image_into_numpy_array(image):
    """This function returns the image into a numpy array of appropriate dimension"""
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8) if \
        len(np.array(image).shape) == 3 else \
        np.array(image.getdata()).reshape((im_height, im_width, 1)).astype(np.uint8)
        
def run_inference_for_single_image(image, graph):
    """This function takes an image and a trained graph and returns an output dictionary with 
    number of detections made, detection calsses, detection boxes and detection scores. 
    The user is supposed to filter out the boxes below a minimum threshold"""
    with graph.as_default():
      with tf.Session() as sess:
        # Get handles to input and output tensors
        ops = tf.get_default_graph().get_operations()
        all_tensor_names = {output.name for op in ops for output in op.outputs}
        tensor_dict = {}
        for key in [
            'num_detections', 'detection_boxes', 'detection_scores',
            'detection_classes', 'detection_masks'
        ]:
          tensor_name = key + ':0'
          if tensor_name in all_tensor_names:
            tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                tensor_name)
        if 'detection_masks' in tensor_dict:
          # The following processing is only for single image
          detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
          detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
          # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
          real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
          detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
          detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
          detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              detection_masks, detection_boxes, image.shape[0], image.shape[1])
          detection_masks_reframed = tf.cast(
              tf.greater(detection_masks_reframed, 0.1), tf.uint8)
          # Follow the convention by adding back the batch dimension
          tensor_dict['detection_masks'] = tf.expand_dims(
              detection_masks_reframed, 0)
        image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

        # Run inference
        output_dict = sess.run(tensor_dict,
                               feed_dict={image_tensor: np.expand_dims(image, 0)})

        # all outputs are float32 numpy arrays, so convert types as appropriate
        output_dict['num_detections'] = int(output_dict['num_detections'][0])
        output_dict['detection_classes'] = output_dict[
            'detection_classes'][0].astype(np.uint8)
        output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
        output_dict['detection_scores'] = output_dict['detection_scores'][0]
        if 'detection_masks' in output_dict:
          output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

#### Test of the pipeline

In [42]:
# sys.path.append("/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages")
import pandas as pd
import glob
dict1 = {'CaptureEventID':['ASG0002tcc', 'ASG0017xvs'],
      'Species': ['giraffe', 'elephant'],
      'DateTime': ['2010-09-20', '9/12/2013 18:26'],
      'SiteID': ['S0', 'M1'],
      'NumSpecies': [1, 1],
      'Count': [1, 1],
      'urls': ['http://www.snapshotserengeti.org/subjects/standard/50c212718a607540b902010c_0.jpg',
               'http://www.snapshotserengeti.org/subjects/standard/5331e2111bccd304b6089b2d_0.JPG'
              ]
               }
df1 = pd.DataFrame(dict1)

dataset = df1
image_name_index=0
url_col_index=6
outpath = '/Users/manishrai/Desktop/outdir_test/'

get_images_from_url(dataset, image_name_index, url_col_index, outpath)

# Model info
PATH_TO_FROZEN_GRAPH = '/Users/manishrai/Desktop/UMN/Research/Zooniverse/camera-trap-detection/my_workspace/training_demo/trained-inference-graphs/output_inference_graph/frozen_inference_graph.pb'
PATH_TO_LABELS = os.path.join('/Users/manishrai/Desktop/UMN/Research/Zooniverse/camera-trap-detection/my_workspace/training_demo/annotations/', 'label_map_focus.pbtxt')
NUM_CLASSES = 1
TEST_IMAGE_PATHS = glob.glob('/Users/manishrai/Desktop/outdir_test/*.jpg')
min_threshold = 0.9
bb_outpath = '/Users/manishrai/Desktop/test_dir/'
PATH_TO_BB_HASHMAP = '/Users/manishrai/Desktop/test_dir/bb_hashmap_test.csv'

Processing image: 0
Processing image: 1


In [43]:
TEST_IMAGE_PATHS

['/Users/manishrai/Desktop/outdir_test/ASG0017xvs.jpg',
 '/Users/manishrai/Desktop/outdir_test/ASG0002tcc.jpg']

In [44]:
run_inference_graph_images(PATH_TO_FROZEN_GRAPH, PATH_TO_LABELS, \
                               NUM_CLASSES, TEST_IMAGE_PATHS, min_threshold, \
                               bb_outpath, PATH_TO_BB_HASHMAP)

{'ASG0017xvs.jpg': {'detection_boxes': array([[0.        , 0.36793044, 1.        , 0.99240124],
         [0.        , 0.00238551, 1.        , 0.4733076 ]], dtype=float32),
  'detection_scores': array([0.99947053, 0.9561418 ], dtype=float32)},
 'ASG0002tcc.jpg': {'detection_boxes': array([[0.30239972, 0.23177734, 0.503748  , 0.40976784]], dtype=float32),
  'detection_scores': array([0.9998913], dtype=float32)}}

<Figure size 600x421 with 0 Axes>

<Figure size 600x421 with 0 Axes>